In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("총 글자 수:", len(raw_text))
print(raw_text[:99]) # 첫 99글자 출력

총 글자 수: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [6]:
import re
text = "Hello, World! Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [token for token in result if token.strip() != '']
print(result)

['Hello', ',', 'World', '!', 'Is', 'this', '--', 'a', 'test', '?']


In [8]:
import re
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [token for token in preprocessed if token.strip() != '']
print(len(preprocessed))
print(preprocessed[:30])  # 첫 30개 토큰 출력

4690
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [10]:
all_tokens = sorted(set(preprocessed))
vocab_size = len(all_tokens)
print("어휘 사전 크기:", vocab_size)

vocab={token: idx for idx, token in enumerate(all_tokens)}
for i, token in enumerate(vocab.items()):
    print(token)
    if i>=50:
        break

어휘 사전 크기: 1130
('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [22]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_id = vocab
        self.id_to_str = {idx: token for token, idx in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [token.strip() for token in preprocessed if token.strip()]
        ids = [self.str_to_id[token] for token in preprocessed]
        return ids
    
    def decode(self, ids):
        text = ' '.join([self.id_to_str[idx] for idx in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text
    # tokens -> id : 인코딩
    # id -> tokens : 디코딩

In [26]:
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know,"
        Mrs.Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print("인코딩 결과:", ids)
print("디코딩 결과:", tokenizer.decode(ids))

인코딩 결과: [1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]
디코딩 결과: " It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [31]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token: idx for idx, token in enumerate(all_tokens)}
print("확장된 어휘 사전 크기:", len(vocab))

for i, token in enumerate(list(vocab.items())[-5:]):
    print(token)
    if i >= 55:
        break

확장된 어휘 사전 크기: 1132
('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [33]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_id = vocab
        self.id_to_str = {idx: token for token, idx in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [
            token.strip() for token in preprocessed if token.strip()
            ]
        preprocessed = [
            token if token in self.str_to_id else "<|unk|>"
            for token in preprocessed
            ]
        ids = [self.str_to_id[token] for token in preprocessed]
        return ids
    
    def decode(self, ids):
        text = ' '.join([self.id_to_str[idx] for idx in ids])
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text
    # tokens -> id : 인코딩
    # id -> tokens : 디코딩

In [37]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join([text1, text2])
print("원문:", text)
tokenizer2 = SimpleTokenizerV2(vocab)
print("토큰 인코딩:", tokenizer2.encode(text))
print("토큰 디코딩:", tokenizer2.decode(tokenizer2.encode(text)))

원문: Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.
토큰 인코딩: [1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]
토큰 디코딩: <|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


In [38]:
from importlib.metadata import version
import tiktoken
print("tiktoken 버전:", version("tiktoken"))

tiktoken 버전: 0.12.0


In [41]:
tokenizer_bpe = tiktoken.get_encoding("gpt2")
text = ("Hello, do you like tea? <|endoftext|> In the sunlit terraces"
        " of someunknownPlace."
)

integers = tokenizer_bpe.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

strings = tokenizer_bpe.decode(integers)
print(strings)


[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]
Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace.
